In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, scale
from sklearn.decomposition import PCA
from sklearn.utils import resample

In [27]:
df = pd.read_csv('./baseline_data.csv',header=0)

In [28]:
df.head()

,area.1002,area.1003,area.1005,area.1006,area.1007,area.1008,area.1009,area.1011,area.1012,area.1013,...,Image.Data.ID,DX.bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,MMSE,imputed_genotype
0,612.577638,2514.366512,1652.050796,731.718725,3794.131041,5012.960558,4951.994756,5666.361234,2568.454160,3147.024829,...,45108,CN,84.8,Male,18,Not Hisp/Latino,White,1,28,True
1,735.292087,2435.629408,1947.966106,721.230657,4101.035394,4469.814924,4002.936490,5427.875646,2778.342103,3907.628206,...,45117,CN,76.3,Female,16,Not Hisp/Latino,White,0,29,True
2,1080.976588,2190.801306,1613.620315,636.078912,5146.969073,6192.609394,4640.889149,6854.023127,3282.902615,3677.408865,...,48617,AD,77.5,Male,12,Not Hisp/Latino,White,2,22,False
3,840.850798,2293.601605,1599.807666,729.344575,3351.924971,4231.417941,3991.795466,5047.106646,2599.222056,3339.590461,...,40683,CN,89.6,Female,16,Not Hisp/Latino,White,0,30,True
4,592.882184,1827.195664,1479.821407,535.558408,3459.934118,5063.103074,3583.954659,4120.492969,2462.352998,2737.467877,...,40708,LMCI,65.1,Female,16,Not Hisp/Latino,White,1,27,True


In [29]:
Catdata = df.loc[0:,'PTETHCAT':'APOE4'].join(df['PTGENDER']).join(df['imputed_genotype'])
Numdata = df.loc[0:,:'Thickness..thickinthehead..2035'].join(df['AGE']).join(df['PTEDUCAT']).join(df['MMSE'])
labels = df['DX.bl']

In [30]:
for i in Catdata.columns:
    the_value = str(Catdata[i].mode().values[0])
    Catdata[i].replace('NaN',the_value,inplace = True)
    dummy_data = pd.get_dummies(Catdata[i], prefix=i+"_", drop_first=True)
    Catdata = pd.concat([Catdata, dummy_data], axis=1)
    Catdata.drop(i, axis=1, inplace=True)
#for i in Numdata.columns:
    #the_value = str(Numdata[i].median().values[0])
    #Numdata[i].replace('NaN',the_value,inplace = True)

In [31]:
le = LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
labels = labels.astype('float')

In [32]:
data = Numdata.join(Catdata)
#data['labels'] = labels
#data = resample(data,n_samples=628, random_state = 1)
#pca = PCA(n_components = 400)
#data = pca.fit_transform(data)

In [33]:
data.shape

(628, 2161)

In [36]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = ([],[],[],[])
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 2018)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = scaler.transform(X_test)
X_train['labels'] = np.array(y_train)

# split, normalize and bootstrap

In [58]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = ([],[],[],[])
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,:'imputed_genotype__True'], data['labels'], test_size = 0.2, random_state = 2018)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = scaler.transform(X_test)
X_train['labels'] = np.array(y_train)

In [37]:
from sklearn.utils import resample
Xs_train = []
Labels = []
for _ in range(5):
    d = resample(X_train,n_samples=502)
    Xs_train.append(d.iloc[:,:-1])
    Labels.append(d['labels'])

# normalize and split

In [10]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = ([],[],[],[])
scaler = StandardScaler()
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 1)
#DATA = X_train
#DATA['labels'] = y_train

In [11]:
from sklearn.utils import resample
Xs_train = []
Labels = []
for _ in range(5):
    d = resample(X_train,n_samples=502)
    Xs_train.append(d.iloc[:,:-1])
    Labels.append(d['labels'])

KeyError: 'labels'

# Gaussian Process Regression

A task that was considerably more difficult was the prediction of MMSE, which itself is a very noisy measure [8]. We used Gaussian Process Regression[5] (GPR) to predict this measure using selected imaging features, demographic features as well as genetic features. The choice of GPR was motivated by the ability of such models to fit very noisy data [4].

In [40]:
from sklearn.gaussian_process import GaussianProcessClassifier
#gp = GaussianProcessClassifier(optimizer='Welch',multi_class='one_vs_rest',n_restarts_optimizer=5)
#gp = LogisticRegression()
#gp = RandomForestClassifier(min_samples_split = 10, n_estimators = 250, max_features = None , min_samples_leaf= 5, max_depth= 10, bootstrap = True)

In [46]:
gpt = []
for i in range(5):
    gp.fit(Xs_train[i],Labels[i])
    gpt.append(gp.predict(X_test))

/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of

In [47]:
gpc = []
for i in range(len(y_test)):
    p = []
    for j in gpt:
        p.append(j[i])
    counts = np.bincount(p)
    mode = np.argmax(counts)
    gpc.append(mode)

In [13]:
from sklearn.metrics import f1_score,accuracy_score

In [48]:
f1_score(y_test,gpc,average='macro')

0.4795708278961867

In [49]:
accuracy_score(y_test,gpc)

0.49206349206349204

In [65]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
pre = lr.predict(X_test)

In [66]:
f1_score(y_test,pre,average='macro')

0.4631339031339032

In [67]:
accuracy_score(y_test,pre)

0.5

In [140]:
model = BaggingClassifier(base_estimator=gp, n_estimators=10, random_state=7, max_samples= 500, max_features = 400,
                         bootstrap=True, bootstrap_features=True)
model.fit(X_train.iloc[:,:-1],y_train)
pre = model.predict(X_test)

In [141]:
print(accuracy_score(y_test,pre))
print(f1_score(y_test,pre,average='macro'))

0.3968253968253968
0.3347519612768283


In [166]:
from sklearn.svm import SVC
model = SVC(C =10,kernel="linear", probability=True)
#model = BaggingClassifier(base_estimator=my_svm, n_estimators=10, random_state=7, max_samples= 500, max_features = 400,bootstrap=True, bootstrap_features=True)
model.fit(X_train.iloc[:,:-1],y_train)
pre = model.predict(X_test)

In [167]:
print(accuracy_score(y_test,pre))
print(f1_score(y_test,pre,average='macro'))

0.46825396825396826
0.42973856209150324


In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss, f1_score 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier


In [45]:
# Voting Ensemble for Classification
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score 
seed = 7
num_trees = 70
#kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
model1 = LogisticRegression(random_state=0, C = 2.5)
estimators.append(('logistic', model1))
#model2 = DecisionTreeClassifier()
#estimators.append(('cart', model2))
model3 = LinearSVC(C = 100,penalty='l2', loss='squared_hinge', dual=False)
estimators.append(('svm', model3))
#model4 = GaussianProcessClassifier(optimizer='Welch',multi_class='one_vs_rest',n_restarts_optimizer=5)
#estimators.append(('GPC', model4))
model5 = RandomForestClassifier(min_samples_split = 12, n_estimators = 250, max_features = None , min_samples_leaf= 5, max_depth= 10, bootstrap = True)
estimators.append(('RF', model5))
# create the ensemble model
gp = VotingClassifier(estimators)
'''ensemble.fit(X_train,y_train)
#print("",ensemble.score(X_test,y_test))
#predictions_train = ensemble.predict(X_train)
predictions_test = ensemble.predict(X_test)
f1_Score=f1_score(y_test, predictions_test, average='macro')
accuracy=accuracy_score(y_test, predictions_test)
print("F1 Score {}".format(f1_Score))
print("Accuracy {}".format(accuracy))
'''
#results = model_selection.cross_val_score(ensemble, X_test, y_test, cv=kfold, scoring='accuracy')
#results = model_selection.cross_val_score(ensemble, principalDf,labels, cv=kfold, scoring='accuracy')
#print(results.mean())
#print("Accuracy: ", results.mean())

'ensemble.fit(X_train,y_train)\n#print("",ensemble.score(X_test,y_test))\n#predictions_train = ensemble.predict(X_train)\npredictions_test = ensemble.predict(X_test)\nf1_Score=f1_score(y_test, predictions_test, average=\'macro\')\naccuracy=accuracy_score(y_test, predictions_test)\nprint("F1 Score {}".format(f1_Score))\nprint("Accuracy {}".format(accuracy))\n'

In [25]:
model5 = RandomForestClassifier(min_samples_split = 10, n_estimators = 250, max_features = None , min_samples_leaf= 5, max_depth= 10, bootstrap = True)
model5.fit(X_train,y_train)
pre = model5.predict(X_test)

In [14]:
print(accuracy_score(y_test,pre))
print(f1_score(y_test,pre,average='macro'))

0.6587301587301587
0.5915851921238022
